### Install dependenies

In [ ]:
import os
import sys
root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)
!{sys.executable} -m pip install numpy pandas intervaltree matplotlib

import numpy as np
import pandas as pd

### Prepare data

In [ ]:
from strategy import PoolData, Pool, Token, Fee, Frequency
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)
data = PoolData.from_pool(pool, Frequency.DAY)
data.plot()

### Define strategy

In [ ]:
from strategy import Position, AbstractStrategy 
from datetime import datetime
from typing import Callable

class ConstantStrategy(AbstractStrategy):        
    def rebalance(
        self,
        t: datetime,
        c: float,
        vol: float,
        l: Callable[[float], float],
        pool_data: PoolData,
    ) -> bool:
        if not self.portfolio.position("main"):
            self.portfolio.add_position(Position(id="main", a = c / 2, b = c * 2))
            pos = self.portfolio.position("main")
            pos.deposit(c, 1 / 2 / c, 1 / 2, pool_data.pool.fee.percent)

        return False

### Backtest

In [ ]:
from strategy import Backtest, Position

backtest = Backtest(ConstantStrategy)
backtest.run(data, 0.001)
backtest.plot()

In [ ]:
from strategy import Portfolio

p = Portfolio()
pos = Position(id="main", a = 16 / 2, b = 16 * 2)
p.add_position(pos)
pos.deposit(16, 1 / 2 / 16, 1 / 2, 0.003)
pos.rebalance(16 / 3, 16 * 3, 16, 0.003)
# backtest.history
data.swap_prices("2021-05-05", "2021-05-06")

In [ ]:
backtest.history["c"][0]